In [4]:
import gym
env = gym.make('HalfCheetah-v2')

In [5]:
print(env.observation_space.shape, env.action_space.shape)

(11,) (2,)


In [27]:
obs = env.reset()
print(obs)
print(env.sim.data.qpos)
print(env.sim.data.qvel)

[ 0.99851491  0.99994071  0.05447909 -0.01088972 -0.0978359  -0.13780612
  0.00244727  0.00189328  0.30758278  0.14805028  0.        ]
[ 0.05450607 -0.01088994 -0.0978359  -0.13780612]
[0.00244727 0.00189328 0.         0.        ]


In [28]:
st = env.sim.get_state()
print(st)
print(env.sim.data.qpos)
print(env.sim.data.qvel)

MjSimState(time=0.0, qpos=array([ 0.05450607, -0.01088994, -0.0978359 , -0.13780612]), qvel=array([0.00244727, 0.00189328, 0.        , 0.        ]), act=None, udd_state={})
[ 0.05450607 -0.01088994 -0.0978359  -0.13780612]
[0.00244727 0.00189328 0.         0.        ]


In [29]:
obs = env.reset()
print(obs)
print(env.sim.data.qpos)
print(env.sim.data.qvel)

[ 9.98816012e-01  9.98525495e-01  4.86474573e-02 -5.42847717e-02
  5.68376166e-02 -1.31636684e-01 -1.72325536e-05  3.22611473e-03
  1.53042232e-01  1.35880505e-01  0.00000000e+00]
[ 0.04866667 -0.05431147  0.05683762 -0.13163668]
[-1.72325536e-05  3.22611473e-03  0.00000000e+00  0.00000000e+00]


In [30]:
env.sim.set_state(st)
print(env.sim.data.qpos)
print(env.sim.data.qvel)

[ 0.05450607 -0.01088994 -0.0978359  -0.13780612]
[0.00244727 0.00189328 0.         0.        ]


In [6]:
for i in range(6):
    env.reset()
    rew = 0
    
    while True:
        _, r, done, _ = env.step(env.action_space.sample())
        
        rew += r
        
        if done==True:
            print('Ep %d: %.2f' % (i+1, rew))
            
            break

Ep 1: -48.75
Ep 2: -44.70
Ep 3: -39.31
Ep 4: -45.03
Ep 5: -37.28
Ep 6: -40.89


In [1]:
import torch as T
import torch.nn as nn

In [2]:
state_size = 24
compare = 5
print((-compare*T.ones(state_size) == -compare*T.ones(state_size)))

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       dtype=torch.uint8)


In [1]:
import gym
import math
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [2]:
env = gym.make('MountainCarContinuous-v0')
env.seed(101)
np.random.seed(101)

In [3]:
print('observation space:', env.observation_space)
print('action space:', env.action_space)
print('  - low:', env.action_space.low)
print('  - high:', env.action_space.high)

observation space: Box(2,)
action space: Box(1,)
  - low: [-1.]
  - high: [1.]


## get mujoco videos

In [179]:
from envs import video_wrapper

In [180]:
import gym
env_names = ['HalfCheetah-v2', 'Hopper-v2', 'Ant-v2', 'Reacher-v2']
env_id = 0
env_name = env_names[env_id]

env = gym.make(env_name)
print(env.observation_space.shape, env.action_space.shape)
dX = env.observation_space.shape[0]
dU = env.action_space.shape[0]
print(dX, dU)

# env.reset()
# for _ in range(1000):
#    env.render()
#    env.step(env.action_space.sample()) # take a random action

for i in range(6):
    env.reset()
    rew = 0
    
    while True:
        _, r, done, _ = env.step(env.action_space.sample())
        
        rew += r
        
        if done==True:
            print('Ep %d: %.2f' % (i+1, rew))
            
            break

(17,) (6,)
17 6
Ep 1: -223.48
Ep 2: -242.45
Ep 3: -169.68
Ep 4: -255.72
Ep 5: -339.02
Ep 6: -225.63


In [207]:
import pickle

trajs_demo, demo_st_info, demo_rew_info = pickle.load(open('Demo/my_demo_'+env_name+'_bc_sac_4000+.pkl', 'rb'))
print(len(trajs_demo), len(trajs_demo[0]))
# print(trajs_demo[0][0][0].dtype)
print(trajs_demo[0][0][0])
print(trajs_demo[0][0][1])
print(trajs_demo[0][1][0])
print(trajs_demo[0][1][1])
print(trajs_demo[0][2][0])
print(trajs_demo[0][2][1])
print(trajs_demo[0][3][0])
print(trajs_demo[0][3][1])
print(demo_st_info[0][0])
print(demo_st_info[1][0])

demo_observations = []
demo_actions = []
for traj in trajs_demo:
    for dat in traj:
        obs, act, new_obs, _ = dat
        demo_observations.append(obs)
        demo_actions.append(act)
        
print('number of demo observations: ', len(demo_observations))
print('number of demo actions: ', len(demo_actions))    

demo_O_mean = np.mean(demo_observations, axis=0)
demo_O_std = np.std(demo_observations, axis=0)
demo_A_mean = np.mean(demo_actions, axis=0)
demo_A_std = np.std(demo_actions, axis=0)
print('mean of demo observations: ',demo_O_mean)
print('std of demo observations: ', demo_O_std)
print('mean of demo actions: ',demo_A_mean)
print('std of demo actions: ', demo_A_std)

100 1000
[-0.05938697  0.08183036  0.02458883 -0.01742797 -0.01609161  0.04450291
 -0.07814357 -0.09891209 -0.04066111 -0.05090073 -0.0017749   0.03683974
  0.06899738  0.14454107  0.06860361 -0.10913674 -0.03406164]
[-0.639646    1.0556277  -1.0177609  -0.44709456 -0.6476692  -0.97560084]
[-0.09540331  0.06572465 -0.2427492   0.41544835 -0.37869978 -0.10846443
 -0.17088567 -0.28335864 -0.82602429 -1.02507544 -0.23427068 -6.45173848
  9.3182296  -8.79654918 -4.56252646 -2.7529794  -5.25726898]
[-0.1439763   0.8596165  -1.194109   -0.75786155 -0.7505367  -0.87685627]
[-0.14546418  0.10082363 -0.2776329   0.52300003 -0.43256342 -0.45104482
 -0.32563859 -0.495685   -0.19347363 -1.03764489  1.14007219  2.85764617
 -1.28704863  0.91818254 -7.22550572 -3.20821415 -3.14600408]
[ 0.24971947 -0.7774035   0.47915685 -0.98256767 -0.81378365 -0.88366234]
[ -0.18487895   0.2105282    0.11297773  -0.23240031   0.10376701
  -0.78604651  -0.47398773  -0.50238539   0.4396503   -1.03999262
   1.77842538

In [201]:
# class TransAction:
#     def __init__(self, env, a, b):
#         self.env = env
#         self.action_space = self.env.action_space
#         self.observation_space = self.env.observation_space
#         self.reward_range = self.env.reward_range
#         self.metadata = self.env.metadata
        
#         self.a = np.zeros(env.action_space.shape, dtype=env.action_space.dtype) + a
#         self.b = np.zeros(env.action_space.shape, dtype=env.action_space.dtype) + b
        
#     def clip_action(self, action):
#         return np.clip(action, self.action_space.low, self.action_space.high)
    
#     def rescale_action(self, action):
#         # print(action)
#         assert np.all(np.greater_equal(action, self.a)), (action, self.a)
#         assert np.all(np.less_equal(action, self.b)), (action, self.b)
#         low = self.env.action_space.low
#         high = self.env.action_space.high
#         # print(low, high, self.b, self.a)
#         action = low + (high - low)*((action - self.a)/(self.b - self.a))
#         # print(action)
#         action = np.clip(action, low, high)
#         # print(action)
#         return action

In [202]:
# trans_action = TransAction(env, -1.0, 1.0)

In [212]:
# for i in range(100):
#     rews=0.0
#     rews_ground=np.sum(demo_rew_info[i])
#     cur_st = demo_st_info[i][0]
#     # print(cur_st)
#     env.reset()
#     env.sim.set_state(cur_st) #restore
#     for t in range(1000):
#         act = trajs_demo[i][t][1]
#         # act = trans_action.rescale_action(trans_action.clip_action(act))
#         ground_next_obs, reward, done, _ = env.step(act)
#         # eval_env.step(eval_env.action_space.sample())
#         rews+=reward
#     print(rews)
#     if np.fabs(rews-rews_ground) > 1.0:
#         print(i, rews_ground, rews)
        

In [209]:
eval_dir='videos'
full_vid_name='halfcheetah_bc_sac_4000+_1000step'
eval_env = video_wrapper.VideoWrapper(env, base_path=eval_dir, base_name=full_vid_name,
                                      new_video_every_reset=True)

In [210]:
def sim_get_obs_gym(env_id):
    if env_id == 3:
        theta = eval_env.sim.data.qpos.flat[:2]
        return np.concatenate([
            np.cos(theta),
            np.sin(theta),
            eval_env.sim.data.qpos.flat[2:],
            eval_env.sim.data.qvel.flat[:2],
            eval_env.get_body_com("fingertip") - eval_env.get_body_com("target")
        ])
    elif env_id == 0:
        return np.concatenate([
            eval_env.sim.data.qpos.flat[1:],
            eval_env.sim.data.qvel.flat,
        ])
        

In [211]:
for i in range(3):
    rews=0.0
    cur_st = demo_st_info[i][0]
    print(cur_st)
    eval_env.reset()
    eval_env.sim.set_state(cur_st) #restore
    for t in range(1000):
        ground_next_obs, reward, done, _ = eval_env.step(trajs_demo[i][t][1])
        # eval_env.step(eval_env.action_space.sample())
        rews+=reward
    print(rews)
eval_env.close()

MjSimState(time=0.0, qpos=array([-0.02524417, -0.05938697,  0.08183036,  0.02458883, -0.01742797,
       -0.01609161,  0.04450291, -0.07814357, -0.09891209]), qvel=array([-0.04066111, -0.05090073, -0.0017749 ,  0.03683974,  0.06899738,
        0.14454107,  0.06860361, -0.10913674, -0.03406164]), act=None, udd_state={})
4979.1363259712125
MjSimState(time=0.0, qpos=array([ 0.00871898, -0.06036362, -0.07576941,  0.02058242, -0.08460817,
       -0.06438892,  0.09674049, -0.03903066, -0.05924532]), qvel=array([-0.14778943,  0.14992812, -0.00704264,  0.06968367, -0.00032993,
        0.03796217, -0.00173116,  0.0111537 ,  0.16965625]), act=None, udd_state={})
5267.55737981211
MjSimState(time=0.0, qpos=array([-0.06661751, -0.08316578, -0.09033935,  0.06358277, -0.08679948,
       -0.05625694,  0.08930589,  0.0928403 ,  0.04540023]), qvel=array([-0.04627891,  0.23530191, -0.00210341, -0.05638135, -0.06457223,
        0.06970487, -0.02469187, -0.01243139,  0.15573852]), act=None, udd_state={})
5

## tf_agents

In [37]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import numpy as np

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

tf.compat.v1.enable_v2_behavior()

In [39]:
environment = suite_gym.load('HalfCheetah-v2')
print('action_spec:', environment.action_spec())
print('time_step_spec.observation:', environment.time_step_spec().observation)
print('time_step_spec.step_type:', environment.time_step_spec().step_type)
print('time_step_spec.discount:', environment.time_step_spec().discount)
print('time_step_spec.reward:', environment.time_step_spec().reward)


action_spec: BoundedArraySpec(shape=(6,), dtype=dtype('float32'), name='action', minimum=-1.0, maximum=1.0)
time_step_spec.observation: BoundedArraySpec(shape=(17,), dtype=dtype('float64'), name='observation', minimum=-1.7976931348623157e+308, maximum=1.7976931348623157e+308)
time_step_spec.step_type: ArraySpec(shape=(), dtype=dtype('int32'), name='step_type')
time_step_spec.discount: BoundedArraySpec(shape=(), dtype=dtype('float32'), name='discount', minimum=0.0, maximum=1.0)
time_step_spec.reward: ArraySpec(shape=(), dtype=dtype('float32'), name='reward')


In [42]:
obs = environment.reset()
print(obs)

TimeStep(step_type=array(0, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([ 0.00193556,  0.05883425,  0.0323004 , -0.06682837,  0.08272298,
       -0.0978205 ,  0.0268753 ,  0.05983981,  0.00440393, -0.14672328,
        0.01119457,  0.0645935 ,  0.00141634,  0.03690026, -0.0740658 ,
        0.10998834, -0.15960418]))


In [54]:
environment._env._gym_env.sim.get_state()

MjSimState(time=0.0, qpos=array([ 0.09335548,  0.00193556,  0.05883425,  0.0323004 , -0.06682837,
        0.08272298, -0.0978205 ,  0.0268753 ,  0.05983981]), qvel=array([ 0.00440393, -0.14672328,  0.01119457,  0.0645935 ,  0.00141634,
        0.03690026, -0.0740658 ,  0.10998834, -0.15960418]), act=None, udd_state={})

In [55]:
import sys
sys.version

'3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) \n[GCC 7.2.0]'

### verify action and observation consistence of mujoco_2.0_HalfCheetah-v3 and mujoco_1.5_HalfCheetah-v2

In [172]:
import pickle
file_path = 'Demo/mujoco_2.0_MjSimState_HalfCheetah-v3.pkl'
with open(file_path, 'rb') as f:
    st, act_list, obs_list = pickle.load(f)
    f.close()

In [173]:
print(st)

MjSimState(time=0.0, qpos=array([ 0.03700296, -0.05824359,  0.05125691, -0.03565497,  0.08908716,
        0.07130794, -0.01255094, -0.03211012,  0.05172378]), qvel=array([ 0.24427405,  0.05353578,  0.09666746,  0.06248731, -0.05303575,
        0.00187552,  0.15097784,  0.18083958, -0.03890492]), act=None, udd_state={})


In [174]:
import gym
env = gym.make('HalfCheetah-v2')

In [175]:
env.reset()

array([-0.0573714 , -0.0437987 , -0.00960879, -0.03486889,  0.0057009 ,
        0.08732049,  0.0001938 , -0.09403904,  0.01030172, -0.01722315,
       -0.03887511,  0.03470544, -0.01253835, -0.09650785, -0.07723616,
        0.01993407, -0.02343893])

In [176]:
env.sim.set_state(st)

In [162]:
def sim_get_obs_gym(eval_env, env_id):
    if env_id == 3:
        theta = eval_env.sim.data.qpos.flat[:2]
        return np.concatenate([
            np.cos(theta),
            np.sin(theta),
            eval_env.sim.data.qpos.flat[2:],
            eval_env.sim.data.qvel.flat[:2],
            eval_env.get_body_com("fingertip") - eval_env.get_body_com("target")
        ])
    elif env_id == 0:
        return np.concatenate([
            eval_env.sim.data.qpos.flat[1:],
            eval_env.sim.data.qvel.flat,
        ])
        

In [163]:
print(sim_get_obs_gym(env, 0))

[-0.05824359  0.05125691 -0.03565497  0.08908716  0.07130794 -0.01255094
 -0.03211012  0.05172378  0.24427405  0.05353578  0.09666746  0.06248731
 -0.05303575  0.00187552  0.15097784  0.18083958 -0.03890492]


In [164]:
rew = 0.0
for i in range(1000):
    obs, r, done, _  = env.step(act_list[i])
    rew += r
    print(obs)
    print(obs_list[i][0])
    print((obs==obs_list[i][0]).all())

[-7.26283918e-02  1.07337293e-01 -2.98268612e-01  9.31073966e-02
  4.94793858e-03 -3.19151895e-01 -2.35235729e-01  1.38030877e-01
 -8.46577075e-01 -5.63889422e-01  1.64443365e+00 -6.82705595e+00
 -6.02722822e-01 -1.90362207e+00 -9.06620433e+00 -5.80714033e+00
  2.04424890e+00]
[-7.26283918e-02  1.07337293e-01 -2.98268612e-01  9.31073966e-02
  4.94793858e-03 -3.19151895e-01 -2.35235729e-01  1.38030877e-01
 -8.46577075e-01 -5.63889422e-01  1.64443365e+00 -6.82705595e+00
 -6.02722822e-01 -1.90362207e+00 -9.06620433e+00 -5.80714033e+00
  2.04424890e+00]
True
[-0.12895298  0.10561576 -0.55280705  0.37162245 -0.35756074 -0.46330636
 -0.24948647 -0.13114232  0.18278898 -1.2650177  -0.79832523 -2.44780629
  6.3672916  -7.77419496  1.34348649  2.25999377 -8.47083258]
[-0.12895298  0.10561576 -0.55280705  0.37162245 -0.35756074 -0.46330636
 -0.24948647 -0.13114232  0.18278898 -1.2650177  -0.79832523 -2.44780629
  6.3672916  -7.77419496  1.34348649  2.25999377 -8.47083258]
True
[-1.72029406e-01  

[ -0.03111893  -0.09306878   0.78141441   0.45450047   0.58159662
   0.20771403   0.22869126  -0.14768355  13.0236323    0.25310901
  -0.45897808  11.71408293   6.50188383   4.37027862 -18.04872229
 -16.57560161  -9.71422518]
True
[ -0.06232913   0.10106354   0.87532046   0.60129658   0.11536734
  -1.14348021  -0.43384635  -0.3875587   13.69058048  -0.99527255
   5.34689421  -1.89613308  -0.80942042 -13.16962308 -27.70734832
  -9.38730318  -1.50884168]
[ -0.06232913   0.10106354   0.87532046   0.60129658   0.11536734
  -1.14348021  -0.43384635  -0.3875587   13.69058048  -0.99527255
   5.34689421  -1.89613308  -0.80942042 -13.16962308 -27.70734832
  -9.38730318  -1.50884168]
True
[-5.46481203e-03  3.07904322e-01  2.01731010e-01 -1.47489728e-01
 -3.22430641e-02 -9.16500813e-01 -8.70924590e-01 -4.79797457e-01
  1.41750004e+01  1.40221060e+00  4.58933903e+00 -1.62673159e+01
 -2.04493026e+01  2.14371729e+00  7.26513530e+00 -2.16654103e+00
 -1.21755766e+00]
[-5.46481203e-03  3.07904322e-01  

[-0.05354254  0.10885545 -0.59812632 -0.46187548 -0.42048581  0.63450886
  0.77888849  0.55930497 16.02901871 -1.13546191 -2.73815362  2.6503803
  8.6895259   0.43516621  9.32858379 12.87285468  4.78213545]
[-0.05354254  0.10885545 -0.59812632 -0.46187548 -0.42048581  0.63450886
  0.77888849  0.55930497 16.02901871 -1.13546191 -2.73815362  2.6503803
  8.6895259   0.43516621  9.32858379 12.87285468  4.78213545]
True
[-0.05808341 -0.03206169 -0.02067772  0.07956053  0.2031973   0.70100325
  0.85099634  0.5195211  16.51906679  0.77559086 -1.99925737 14.80789808
 16.19323169  9.03974566 -3.00898362 -6.59337305 -0.62768333]
[-0.05808341 -0.03206169 -0.02067772  0.07956053  0.2031973   0.70100325
  0.85099634  0.5195211  16.51906679  0.77559086 -1.99925737 14.80789808
 16.19323169  9.03974566 -3.00898362 -6.59337305 -0.62768333]
True
[ -0.03955227  -0.11856026   0.81148532   0.52217795   0.57221622
   0.10653204   0.14401621   0.09711549  13.682512    -0.1004857
   0.22023538  11.54924921   

  1.43952727e+01]
[-1.22838605e-02  3.04592440e-01 -4.81368816e-01 -5.75348556e-01
 -4.45132615e-01  5.35744659e-02 -9.09099718e-02  9.60890250e-02
  1.51062063e+01 -1.13097121e+00 -1.32334162e+00 -9.52762829e+00
 -1.33667578e+01  1.16563966e+00  1.81873824e+01  2.71005159e+01
  1.43952727e+01]
True
[-0.06322401  0.13146368 -0.54305542 -0.53525165 -0.39382846  0.7749756
  0.58746722  0.40115945 15.60853475 -0.5849963  -3.48368891  1.20967286
 10.06084182  0.65028697  0.26036302 18.96873711  7.34245348]
[-0.06322401  0.13146368 -0.54305542 -0.53525165 -0.39382846  0.7749756
  0.58746722  0.40115945 15.60853475 -0.5849963  -3.48368891  1.20967286
 10.06084182  0.65028697  0.26036302 18.96873711  7.34245348]
True
[-0.06118516 -0.04159835 -0.05805082  0.12206506  0.21056315  0.70053565
  0.83183512  0.50566373 16.36113024  0.75081624 -2.25289163 13.71657123
 18.20418937  8.88275743 -1.85990293 -5.02088767 -1.72942738]
[-0.06118516 -0.04159835 -0.05805082  0.12206506  0.21056315  0.70053565

[ -0.03049583   0.29679433   0.18361775  -0.05704522   0.03088705
  -0.88391699  -0.83666687  -0.54148306  14.44519271   0.88157422
   3.94220932 -16.77341217  -9.11708702 -14.39457526   7.33302852
  -3.00268668  -2.76261277]
[ -0.03049583   0.29679433   0.18361775  -0.05704522   0.03088705
  -0.88391699  -0.83666687  -0.54148306  14.44519271   0.88157422
   3.94220932 -16.77341217  -9.11708702 -14.39457526   7.33302852
  -3.00268668  -2.76261277]
True
[ -0.03575194   0.31415165  -0.44208101  -0.61796455  -0.4577445
   0.09294964  -0.03467443  -0.14564049  15.04457879  -1.40746465
  -1.02421065 -10.90343312  -9.89364777  -0.40673733  17.93322434
  26.12343327  13.90671556]
[ -0.03575194   0.31415165  -0.44208101  -0.61796455  -0.4577445
   0.09294964  -0.03467443  -0.14564049  15.04457879  -1.40746465
  -1.02421065 -10.90343312  -9.89364777  -0.40673733  17.93322434
  26.12343327  13.90671556]
True
[-0.07777581  0.15386404 -0.53659057 -0.54360643 -0.41764909  0.73667571
  0.63074809  0

 -7.96359537e+00]
[-3.31457942e-02 -1.75829988e-01  8.13762500e-01  5.66577299e-01
  6.55341874e-01  1.38130583e-01  1.59573293e-01  1.51944955e-01
  1.33977392e+01  1.01047857e-02  5.90391338e-01  1.08972558e+01
  3.64637846e+00  6.16341517e+00 -1.84201966e+01 -1.65306938e+01
 -7.96359537e+00]
True
[ -0.06518951   0.05748812   0.87990398   0.27833276   0.6901021
  -1.13909939  -0.41254133  -0.3808603   14.55177711  -0.85612137
   5.8306363   -1.95631226 -10.2065161   -2.7554594  -25.30828345
  -7.52924093 -10.3546295 ]
[ -0.06518951   0.05748812   0.87990398   0.27833276   0.6901021
  -1.13909939  -0.41254133  -0.3808603   14.55177711  -0.85612137
   5.8306363   -1.95631226 -10.2065161   -2.7554594  -25.30828345
  -7.52924093 -10.3546295 ]
True
[-2.21651338e-02  2.63662109e-01  2.56359676e-01 -1.47886693e-01
  2.38627889e-03 -9.08207236e-01 -8.29356104e-01 -5.31128383e-01
  1.42092600e+01  9.29355198e-01  4.13975803e+00 -1.74250982e+01
 -8.05361706e+00 -1.52790807e+01  7.69267324e+00 

In [165]:
print(rew)

13230.697763216693


In [177]:
traj = []
traj_st = []
traj_rew = []
rew = 0

traj_st.append(st)
for t in range(1000):
    act = act_list[t]
    
    new_obs, r, done, _ = env.step(act)

    traj.append([obs, act, new_obs])
    traj_rew.append(r)
    
    obs = new_obs
    rew += r
    st = env.sim.get_state()
    traj_st.append(st)

print(rew)

13230.697763216693
